## IMDB Review NLP

--- 

Neste notebook vamos ler um arquivo do tipo **.csv** com cerca de 50.000 reviews de diversos filmes do [IMDB](https://www.imdb.com/) disponíveis no [Kaggle](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) e analisar o sentimento de cada review.

---

In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pds
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as ptly

from bs4 import BeautifulSoup

In [ ]:
# reads the csv using the pandas library
data_frame = pds.read_csv('../assets/Reviews/reviews.csv')
# defines a pattern to search inside the csv
pattern = r'(<br>|<br />|http)'
# replaces the pattern values with empty string
data_frame['review'] = data_frame['review'].str.replace(pattern, "", regex=True)


In [ ]:
# init chart view
figure, axes = plt.subplots()
# counting values
count = pds.value_counts(data_frame['sentiment'])
# chart data config
sentiments = ['Negativo', 'Positivo']
counts = [count['negative'], count['positive']]
bar_labels = ['Negativo', 'Positivo']
bar_colors = ['tab:red', 'tab:blue']
# axes config
axes.bar(sentiments, counts, label=bar_labels, color=bar_colors)
axes.set_ylabel('Ocorrência')
axes.set_title('Distribuição de Sentimentos')
# chart show
plt.show()